# Getting Started with fastcpd: Part 3 - Evaluation and Visualization

This tutorial shows how to evaluate detection results and create visualizations.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fastcpd import fastcpd
from fastcpd.datasets import make_mean_change
from fastcpd.metrics import precision_recall, evaluate_all
from fastcpd.visualization import plot_detection

%matplotlib inline

## Complete Example: Generate → Detect → Evaluate

In [ ]:
# Step 1: Generate data
data_dict = make_mean_change(n_samples=500, n_changepoints=3, seed=42)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Step 2: Detect change points
result = fastcpd(data, family="mean", beta="MBIC")
detected_cps = result.cp_set

print("True change points:    ", true_cps)
print("Detected change points:", detected_cps)

## Basic Evaluation Metrics

### Precision and Recall

- **Precision**: What fraction of detected CPs are correct?
- **Recall**: What fraction of true CPs were found?

In [ ]:
# Evaluate with tolerance margin of 10 samples
metrics = precision_recall(true_cps, detected_cps, margin=10)

print(f"Precision: {metrics['precision']:.3f}")
print(f"Recall:    {metrics['recall']:.3f}")
print(f"F1 Score:  {metrics['f1_score']:.3f}")
print(f"\nTrue Positives:  {metrics['true_positives']}")
print(f"False Positives: {metrics['false_positives']}")
print(f"False Negatives: {metrics['false_negatives']}")

## Comprehensive Evaluation

Use `evaluate_all()` to get all metrics at once.

In [ ]:
# Get all metrics
all_metrics = evaluate_all(true_cps, detected_cps, n_samples=500, margin=10)

# Print summary
print(all_metrics['summary'])

In [ ]:
# Access individual metric groups
print("\nPoint Metrics:")
for key, value in all_metrics['point_metrics'].items():
    print(f"  {key}: {value}")

print("\nDistance Metrics:")
for key, value in all_metrics['distance_metrics'].items():
    if isinstance(value, (int, float)):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

## Visualization

### Basic Plot

In [ ]:
# Simple visualization
plt.figure(figsize=(14, 5))
plt.plot(data, linewidth=0.8, label='Data', color='black', alpha=0.7)

# True change points
for cp in true_cps:
    plt.axvline(cp, color='green', linestyle='--', linewidth=2, alpha=0.7, 
                label='True CP' if cp == true_cps[0] else '')

# Detected change points
for cp in detected_cps:
    plt.axvline(cp, color='red', linestyle=':', linewidth=2.5, alpha=0.8,
                label='Detected CP' if cp == detected_cps[0] else '')

plt.xlabel('Time', fontsize=12)
plt.ylabel('Value', fontsize=12)
plt.title('Change Point Detection Results', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Built-in Visualization with Metrics

In [ ]:
# Create plot with metrics overlay
fig, axes = plot_detection(
    data, 
    true_cps, 
    detected_cps,
    metric_result=all_metrics,
    title="Mean Change Detection with Metrics"
)
plt.show()

## Interpreting Results

### What makes a good detection?

**Perfect detection:**
- Precision = 1.0 (no false positives)
- Recall = 1.0 (no missed change points)
- F1 Score = 1.0

**Over-detection:**
- Precision < 1.0 (too many false alarms)
- Recall = 1.0 (found all true CPs)
- Many detected CPs

**Under-detection:**
- Precision = 1.0 (all detections correct)
- Recall < 1.0 (missed some true CPs)
- Few detected CPs

## Different Scenarios

### Scenario 1: Perfect Detection

In [ ]:
true_cps = [100, 200, 300]
detected_cps = [100, 200, 300]

metrics = precision_recall(true_cps, detected_cps, margin=10)
print(f"Precision: {metrics['precision']:.3f}")
print(f"Recall:    {metrics['recall']:.3f}")
print(f"F1 Score:  {metrics['f1_score']:.3f}")

### Scenario 2: Over-detection (False Positives)

In [ ]:
true_cps = [100, 200, 300]
detected_cps = [100, 150, 200, 250, 300]  # Extra CPs at 150 and 250

metrics = precision_recall(true_cps, detected_cps, margin=10)
print(f"Precision: {metrics['precision']:.3f}  (3 correct out of 5)")
print(f"Recall:    {metrics['recall']:.3f}  (found all 3 true CPs)")
print(f"False Positives: {metrics['unmatched_pred']}")

### Scenario 3: Under-detection (Missed CPs)

In [ ]:
true_cps = [100, 200, 300, 400]
detected_cps = [100, 200]  # Missed 300 and 400

metrics = precision_recall(true_cps, detected_cps, margin=10)
print(f"Precision: {metrics['precision']:.3f}  (all detections correct)")
print(f"Recall:    {metrics['recall']:.3f}  (found 2 out of 4)")
print(f"Missed CPs: {metrics['unmatched_true']}")

## Tips for Better Detection

### 1. Adjust Beta for Sensitivity

In [ ]:
# Generate data
data_dict = make_mean_change(n_samples=300, n_changepoints=3, seed=42)
data = data_dict['data']
true_cps = data_dict['changepoints']

# Try different beta values
print("True change points:", true_cps)
print("\nDetection with different beta values:")

for beta in ["MBIC", "BIC", 5.0, 15.0]:
    result = fastcpd(data, family="mean", beta=beta)
    metrics = precision_recall(true_cps, result.cp_set, margin=10)
    print(f"\nBeta={str(beta):6s}: {len(result.cp_set)} CPs - "
          f"Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}")
    print(f"           Detected: {result.cp_set}")

### 2. Choose Appropriate Margin

The margin determines how close a detected CP must be to a true CP to count as correct.

In [ ]:
true_cps = [100, 200, 300]
detected_cps = [105, 195, 305]  # Off by 5 samples

for margin in [1, 5, 10, 20]:
    metrics = precision_recall(true_cps, detected_cps, margin=margin)
    print(f"Margin={margin:2d}: Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}")

## Summary

### Quick Evaluation Workflow:

```python
# 1. Detect
result = fastcpd(data, family="mean", beta="MBIC")

# 2. Evaluate
metrics = precision_recall(true_cps, result.cp_set, margin=10)
print(f"Precision: {metrics['precision']:.3f}")
print(f"Recall: {metrics['recall']:.3f}")

# 3. Visualize
plot_detection(data, true_cps, result.cp_set)
```

### Key Metrics:
- **Precision**: Fraction of detected CPs that are correct
- **Recall**: Fraction of true CPs that were found
- **F1 Score**: Harmonic mean of precision and recall

### Available Functions:
- `precision_recall()` - Basic metrics
- `evaluate_all()` - All metrics at once
- `plot_detection()` - Visualization with metrics

---

## What's Next?

- Try different model families (variance, GLM, ARMA, GARCH)
- Experiment with beta values to control sensitivity
- Apply to your own real-world data
- Explore advanced metrics (Hausdorff distance, Adjusted Rand Index)

For more details, see the [documentation](https://github.com/zhangxiany-tamu/fastcpd_Python).